helper functions

In [ ]:


import random, re
import sympy, math
def generatePrime():
    p, q = sympy.randprime(2, 20), sympy.randprime(2, 20)
    while q == p:
        q = sympy.randprime(2, 20)
    return p, q

def gcd(a, b):
    while b != 0:
        c = a % b
        a = b
        b = c
    return a

def modinv(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def coprimes(a,phi):
    l = []
    for x in range(2, a):
        if gcd(a, x) == 1 and modinv(x,phi) != None:
            l.append(x)
    for x in l:
        if x == modinv(x,phi):
            l.remove(x)
    return l


Fiat-Shamir Protocol

In [ ]:


def fs_claimant():
    p,q = generatePrime()
    n = p*q
    s = random.randint(1,n-1)
    v = (s**2)%n
    r = random.randint(1,n-1)
    x = (r**2)%n
    #c = int(input("challenge(0 or 1) :"))
    c = random.randint(0,1)
    y = (r * s**c)%n
    return x,v,c,n,y
    
def fs_verifier(x,v,c,n,y):
    lhs = (y**2)%n
    rhs = (x*(v**c))%n
    print(lhs)
    if lhs == rhs:
        return "correct"
    else:
        return "wrong"

x,v,c,n,y = fs_claimant()       
fs_verifier(x,v,c,n,y)

157


'correct'

Guillou Quis

In [ ]:
def gq_claimant():
    #p,q = generatePrime() #private
    p,q = 11,17
    n = p*q  #public
    phi=int(p-1)*int(q-1)
    co_prime_list = coprimes(int(n),phi)
    print("List of co-primes",co_prime_list)
    #e=random.choice(co_prime_list) #public
    e=13
    #s=random.randint(1,n-1)  # private
    s=15
    v=(modinv(s**e,n))  # public
    #r=random.randint(1,n-1)
    r=3
    x=(r**e)%n
    c = int(input("challenge(1 to {}) :".format(e)))
    #c=random.randint(1,e)
    y=(r*(s**c))%n
    return x,v,c,n,y,e

def gq_verifier(x,v,c,n,y,e):
    ans=((y**e)*(v**c))%n
    print("ans",ans)
    if x==ans:
        print("msg crt")
    else:
        print("wrong msg")

x,v,c,n,y,e = gq_claimant()
gq_verifier(x,v,c,n,y,e)

List of co-primes [3, 7, 9, 13, 19, 21, 23, 27, 29, 37, 39, 41, 43, 47, 53, 57, 59, 61, 63, 67, 69, 71, 73, 81, 83, 87, 89, 91, 93, 97, 101, 103, 107, 109, 113, 117, 123, 127, 131, 133, 137, 139, 141, 147, 149, 151, 157, 161, 163, 167, 169, 171, 173, 177, 179, 181, 183]
challenge(1 to 13) :3
ans 148
msg crt


Feige-Fiat

In [ ]:
import numpy as np
def ff_claimant():
    k = 2 #size of vector s and v
    #p,q = generatePrime()
    p,q = 17,11
    n = p*q
    co_prime_list = coprimes(int(n),n)
    #s = co_prime_list[:k]   #private key
    s = [23,19]
    v = [(modinv(s[i]**2,n)) for i in range(k)]  #public key
    #r = random.randint(1,n-1)
    r = 49
    x = (r**2)%n
    #c = [random.randint(0,1) for i in range(k)] #challenge from verifier
    c = [1,0]
    y = (r * np.prod([s[i]**c[i] for i in range(k)]))%n
    return x,v,c,n,y

def ff_verifier(x,v,c,n,y):
    ans = (y**2 * np.prod([v[i]**c[i] for i in range(len(c))]))%n
    print("ans",ans)

    if x == ans:
        print("msg crt")
    else:
        print("wrong msg")

x,v,c,n,y = ff_claimant()       
ff_verifier(x,v,c,n,y)

ans 157
msg crt


**RSA Digital signature**

In [ ]:

# Function to find gcd 
# of two numbers
def euclid(m, n):
      
    if n == 0:
        return m
    else:
        r = m % n
        return euclid(n, r)
      
      
# Program to find 
# Multiplicative inverse
def exteuclid(a, b):
      
    r1 = a
    r2 = b
    s1 = int(1)
    s2 = int(0)
    t1 = int(0)
    t2 = int(1)
      
    while r2 > 0:
          
        q = r1//r2
        r = r1-q * r2
        r1 = r2
        r2 = r
        s = s1-q * s2
        s1 = s2
        s2 = s
        t = t1-q * t2
        t1 = t2
        t2 = t
          
    if t1 < 0:
        t1 = t1 % a
          
    return (r1, t1)
  
# Enter two large prime
# numbers p and q
p = 1948
q = 1931
n = p * q
Pn = (p-1)*(q-1)
d = 23
print("P ",p," Q ",q)
print("n ",n, " Phi ",Pn)  
print("d ",d)
# Generate encryption key 
# in range 1<e<Pn
key = []
  
for i in range(2, Pn):
      
    gcd = euclid(Pn, i)
      
    if gcd == 1:
        key.append(i)
  
  
# Select an encryption key 
# from the above list
r,e = exteuclid(Pn,d)
print("E ",e)
  
# Obtain inverse of 
# encryption key in Z_Pn
#r, d = exteuclid(Pn, e)
if r == 1:
    e = int(e)
    print("decryption key is: ", e)
      
else:
    print("Multiplicative inverse for\
    the given encryption key does not \
    exist. Choose a different encrytion key ")
   
   
# Enter the message to be sent
M = 100
  
# Signature is created by Alice
S = (M**d) % n
  
# Alice sends M and S both to Bob
# Bob generates message M1 using the
# signature S, Alice's public key e 
# and product n.
M1 = (S**e) % n
# If M = M1 only then Bob accepts
# the message sent by Alice.
print("Message1 ",M," Digital Signture ", S," Decrypted Message ",M1)  

M2 = 50

S1 = (M2**d) % n
  
# Alice sends M and S both to Bob
# Bob generates message M1 using the
# signature S, Alice's public key e 
# and product n.
M3 = (S1**e) % n
print("Message1 ",M2," Digital Signture ", S1," Decrypted Message ",M3) 


if M == M1:
    print("As M = M1, Accept the\
    message sent by Alice")
else:
    print("As M not equal to M1,\
    Do not accept the message\
    sent by Alice ")


if M2 == M3:
    print("As M2 = M3, Accept the\
    message sent by Alice")
else:
    print("As M2 not equal to M3,\
    Do not accept the message\
    sent by Alice ")


P  1948  Q  1931
n  3761588  Phi  3757710
d  23
E  1633787
decryption key is:  1633787
Message1  100  Digital Signture  1820456  Decrypted Message  424920
Message1  50  Digital Signture  666188  Decrypted Message  2668692
As M not equal to M1,    Do not accept the message    sent by Alice 
As M2 not equal to M3,    Do not accept the message    sent by Alice 


In [ ]:
New_M = (M * M2) % n
New_S = (S  * S1) % n
print("M1 * M2 = ", New_M)
print("S1 * S2 = ", New_S)


S_ = (New_M**d) % n
M_ = (S_**e) % n
print(S_, M_)

M1 * M2 =  5000
S1 * S2 =  3639412
3639412 1001396
